In [ ]:
# import sys
# !{sys.executable} -m pip install pip knack>=0.5.3
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import io, os, datetime, requests
from collections import defaultdict
import numpy as np
import pandas as pd
import shapely
import boto3
import geopandas as gpd
import io

In [ ]:
import geemap
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'
OUTFILE_NAME = 'GHG-1-ghg-emissions.csv'

In [ ]:
YEARS = [2000, 2020]
DO_UNIT = False

In [ ]:
##### GWPs from AR5 https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter08_FINAL.pdf

SPECIES_INFO = {
    'bc': {
        'name': 'black carbon',
        'filename_suffix': 'BC',
        'ghg': True,
        'gwp20': 460,
        'gwp100': 1600 * 0.94
        # Fuglestvedt et al. "global" values for GWPs
    },
    'ch4': {
        'name': 'methane',
        'filename_suffix': 'CH4',
        'ghg': True,
        'gwp20': 84,
        'gwp100': 28
    },
    'co': {
        'name': 'carbon monoxide',
        'filename_suffix': 'CO',
        'ghg': True,
        'gwp20': 7.65,
        'gwp100': 2.65 * 0.94
        # Midpoints of Fuglestvedt et al. "global" values for GWPs
    },
    'co2': {
        'name': 'carbon dioxide',
        'filename_suffix': '',
        'ghg': True,
        'gwp20': 1,
        'gwp100': 1
    },
#    'fossilco2': {
#        'name': 'carbon dioxide (excl short cycle)',

#    },
    'nox': {
        'name': 'nitrogen oxides',
        'filename_suffix': 'NOx',
        'ghg': True,
        'gwp20': 19,
        'gwp100': -11 * 0.94
        # Fuglestvedt et al. "global" values for GWPs
    },
    'so2': {
        'name': 'sulfur dioxide',
        'filename_suffix': 'SO2',
        'ghg': False
    },
    'oc': {
        'name': 'organic carbon',
        'filename_suffix': 'OC',
        'ghg': True,
        'gwp20': -240,
        'gwp100': -69 * 0.94
        # Fuglestvedt et al. "global" values for GWPs
    },
    'nh3': {
        'name': 'ammonia',
        'filename_suffix': 'NH3',
        'ghg': False
    },
    'nmvoc': {
        'name': 'non-methane volatile organic compounds',
        'filename_suffix': 'NMVOC',
        'ghg': True,
        'gwp20': 14,
        'gwp100': 4.5 * 0.94
        # Fuglestvedt et al. "global" values for GWPs
    }
}

SECTOR_INFO = {
    'agl': {
        'name': 'agriculture livestock',
        'band': 'b1'
    },
    'ags': {
        'name': 'agriculture soils',
        'band': 'b2'
    },
    'awb': {
        'name': 'agriculture waste burning',
        'band': 'b3'
    },
    'ene': {
        'name': 'power generation',
        'band': 'b5'
    },
    'fef': {
        'name': 'fugitives',
        'band': 'b6'
    },
    'ind': {
        'name': 'industry',
        'band': 'b7'
    },
    'res': {
        'name': 'residential, commercial, and other combustion',
        'band': 'b8'
    },
    'shp': {
        'name': 'ships',
        'band': 'b9'
    },
    'slv': {
        'name': 'solvents',
        'band': 'b10'
    },
    'sum': {
        'name': 'all sources',
        'band': 'b11'
    },
    'swd': {
        'name': 'solid waste and wastewater',
        'band': 'b12'
    },
    'tnr': {
        'name': 'off-road transportation',
        'band': 'b13'
    },
    'tro': {
        'name': 'road transportation',
        'band': 'b14'
    }
}

NOX_SECTOR_INFO = {
    'agl': {
        'name': 'agriculture livestock',
        'band': 'b1'
    },
    'ags': {
        'name': 'agriculture soils',
        'band': 'b2'
    },
    'awb': {
        'name': 'agriculture waste burning',
        'band': 'b3'
    },
    'ene': {
        'name': 'power generation',
        'band': 'b4'
    },
    'fef': {
        'name': 'fugitives',
        'band': 'b5'
    },
    'ind': {
        'name': 'industry',
        'band': 'b6'
    },
    'res': {
        'name': 'residential, commercial, and other combustion',
        'band': 'b7'
    },
    'shp': {
        'name': 'ships',
        'band': 'b8'
    },
    'slv': {
        'name': 'solvents',
        'band': 'b9'
    },
    'sum': {
        'name': 'all sources',
        'band': 'b10'
    },
    'swd': {
        'name': 'solid waste and wastewater',
        'band': 'b11'
    },
    'tnr': {
        'name': 'off-road transportation',
        'band': 'b12'
    },
    'tro': {
        'name': 'road transportation',
        'band': 'b13'
    }
}

YEAR_INFO = {
    year: 'b{}'.format(year - 1999) for year in YEARS
}

datasets = defaultdict(None)

In [ ]:
for species in SPECIES_INFO:
    datasets[species] = ee.ImageCollection('users/emackres/CAMS-GLOB-ANTv42_yearly_totalTg{}{}'.format(['', '_'][int(len(SPECIES_INFO[species]['filename_suffix']) > 0)], SPECIES_INFO[species]['filename_suffix']))

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

In [ ]:
def teragramperyear_to_tonneperyear(amt):
    return amt * 1000000

In [ ]:
results = []
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    # AOI
    boundary_id = boundary_id_aoi

    print("\n boundary_id_aoi: " + boundary_id)
    # read boundaries
    boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo).select(['geo_parent_name','geo_name','geo_level','geo_id'])

    all_dfs = {}   # This dict stores all result dfs for single species-year pairs
    for species in SPECIES_INFO:
        if SPECIES_INFO[species]['ghg']:
            print(SPECIES_INFO[species]['name'])
            species_data = datasets[species]
            for year in YEARS:
                print('   {}'.format(year))
                speciesyear_data = species_data.select(YEAR_INFO[year])
                emissions_Tg = speciesyear_data.map(lambda x: x.reduceRegions( boundary_geo_ee, ee.Reducer.sum(), 11131.948796096121, 'epsg:4326'))
                result_df = geemap.ee_to_pandas(emissions_Tg.flatten())  # This df has (numgeoms x numsectors) rows, and 1 data column

                result_df['emissions_CO2e'.format(species, year)] = teragramperyear_to_tonneperyear(result_df['sum']) * SPECIES_INFO[species]['gwp20']
                all_dfs['{0}_{1}'.format(species, year)] = result_df.drop(columns=['sum']).copy()

# reshape so that sector info is in columns instead of rows
    all_spyr_dfs = []  # This stores all complete data rows
    for sp_yr in all_dfs:
        df = all_dfs[sp_yr]
        species, year = sp_yr.split('_')
        geo_features = boundary_geo['features']
        sp_yr_dfs = []  # This stores all correctly shaped dfs for one species-year pair, each df for one geogr feature
        for f in geo_features:
            geo_id = f['properties']['geo_id']
            geo_results = df.loc[df['geo_id'] == geo_id]
            geo_info = geo_results[['geo_parent_name', 'geo_level', 'geo_id', 'geo_name']].iloc[0]
            geo_info_pd = pd.DataFrame(geo_info)
            #geo_info_pd = geo_info_pd.rename({geo_info_pd.index[0]: geo_id}, axis=0)
            georesults_transposed = geo_results.drop(columns=['geo_parent_name', 'geo_level', 'geo_id', 'geo_name']).transpose()
            mapper = {
                 georesults_transposed.columns[j]: '{0}_{1}_{2}_tonnes'.format(species, list(SECTOR_INFO.keys())[j], year) for j in range(len(list(SECTOR_INFO.keys())))
            }
            georesults_transposed_renamed = georesults_transposed.rename(columns=mapper).rename({'emissions_CO2e': geo_id}, axis=0)
            sp_yr_dfs.append(georesults_transposed_renamed)        
        allfeatures_df = pd.concat(sp_yr_dfs, axis=0)
        all_spyr_dfs.append(allfeatures_df)

    result_one_geo = pd.concat([geo_info_pd.transpose().rename(geo_info_pd.transpose()['geo_id'])] + all_spyr_dfs, axis=1)
    for year in YEARS:
        result_one_geo['total_{0}_tonnes'.format(year)] = result_one_geo[['{0}_sum_{1}_tonnes'.format(sp, year) for sp in SPECIES_INFO if sp != 'sum' and SPECIES_INFO[sp]['ghg']]].sum(axis=1)
        result_one_geo = result_one_geo.drop(columns=['{0}_sum_{1}_tonnes'.format(sp, year) for sp in SPECIES_INFO if sp != 'sum' and SPECIES_INFO[sp]['ghg']])

        for sp in SPECIES_INFO:
            if SPECIES_INFO[sp]['ghg']:
                cols_to_sum = []
                for sector in SECTOR_INFO:
                    if not sector == 'sum':
                        cols_to_sum.append(result_one_geo['{0}_{1}_{2}_tonnes'.format(sp, sector, year)].copy())
                result_one_geo['{0}_{1}_tonnes'.format(sp, year)] = pd.concat(cols_to_sum, axis=1).sum(axis=1)
        for sector in SECTOR_INFO:
            if not sector == 'sum':
                cols_to_sum = []
                for sp in SPECIES_INFO:
                    if SPECIES_INFO[sp]['ghg']:
                        cols_to_sum.append(result_one_geo['{0}_{1}_{2}_tonnes'.format(sp, sector, year)].copy())
                result_one_geo['{0}_{1}_tonnes'.format(sector, year)] = pd.concat(cols_to_sum, axis=1).sum(axis=1)
        
        for sp in SPECIES_INFO:
            if SPECIES_INFO[sp]['ghg']:
                result_one_geo['{0}_{1}_co2e'.format(sp, year)] = result_one_geo['{0}_{1}_tonnes'.format(sp, year)] * SPECIES_INFO[sp]['gwp20']
                for sector in SECTOR_INFO:
                    if not sector == 'sum':
                        result_one_geo['{0}_{1}_{2}_co2e'.format(sp, sector, year)] = result_one_geo['{0}_{1}_{2}_tonnes'.format(sp, sector, year)] * SPECIES_INFO[sp]['gwp20']
        big_cols_to_sum = []
        for sector in SECTOR_INFO:
            if not sector == 'sum':
                cols_to_sum = []
                for sp in SPECIES_INFO:
                    if SPECIES_INFO[sp]['ghg']:
                        cols_to_sum.append(result_one_geo['{0}_{1}_{2}_co2e'.format(sp, sector, year)].copy())
                result_one_geo['{0}_{1}_co2e'.format(sector, year)] = pd.concat(cols_to_sum, axis=1).sum(axis=1)
                big_cols_to_sum.append(result_one_geo['{0}_{1}_co2e'.format(sector, year)].copy())
        result_one_geo['total_{0}_co2e'.format(year)] = pd.concat(big_cols_to_sum, axis=1).sum(axis=1)
    
    for unit in ['tonnes', 'co2e']:
        for sp in SPECIES_INFO:
                if SPECIES_INFO[sp]['ghg']:
                    for sector in SECTOR_INFO:
                        if not sector == 'sum':
                            a = result_one_geo['{0}_{1}_{2}_{3}'.format(sp, sector, YEARS[0], unit)]
                            b = result_one_geo['{0}_{1}_{2}_{3}'.format(sp, sector, YEARS[1], unit)]
                            result_one_geo['{0}_{1}_change_{2}'.format(sp, sector, unit)] = ((b - a) / a).fillna('NA')
                    a = result_one_geo['{0}_{1}_{2}'.format(sp, YEARS[0], unit)]
                    b = result_one_geo['{0}_{1}_{2}'.format(sp, YEARS[1], unit)]
                    result_one_geo['{0}_change_{1}'.format(sp, unit)] = ((b - a) / a).fillna('NA')
        if SPECIES_INFO[sp]['ghg']:
            for sector in SECTOR_INFO:
                if not sector == 'sum':
                    a = result_one_geo['{0}_{1}_{2}'.format(sector, YEARS[0], unit)]
                    b = result_one_geo['{0}_{1}_{2}'.format(sector, YEARS[1], unit)]
                    result_one_geo['{0}_change_{1}'.format(sector, unit)] = ((b - a) / a).fillna('NA')
        a = result_one_geo['total_{0}_{1}'.format(YEARS[0], unit)]
        b = result_one_geo['total_{0}_{1}'.format(YEARS[1], unit)]
        result_one_geo['total_change_{0}'.format(unit)] = ((b - a) / a).fillna('NA')
                                       
                                        
                                    
    
    results.append(result_one_geo.copy())
    
    # save to file
    cams_emissions_indicator = pd.concat(results, axis=0)
    cams_emissions_indicator.to_csv(OUTFILE_NAME)

In [ ]:
processedcities = pd.read_csv(OUTFILE_NAME)
# ''out_dir +'/' OUTPUT_FILENAME'
processedcities

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'indicators/'+OUTFILE_NAME+''

processedcities.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')